In [1]:
%load_ext autoreload
%autoreload 2

from statsmodels.base.model import GenericLikelihoodModel

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

from scipy.stats import multivariate_normal

sys.path.append("../")
import vuong_tests4

In [2]:
class JointNormal1(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([params[0], 0.0], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())
    
    
class JointNormal2(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([0.0, params[0]], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())


def setup_shi(yn,xn,return_model=False):
    # model 1 grad, etc.
    nobs = yn.shape[0]
    model1_param = np.array([yn.mean()])
    model2_param = np.array([xn.mean()])
    
    model1_deriv = JointNormal1(yn,xn)
    ll1 = model1_deriv.loglikeobs(model1_param)
    grad1 =  model1_deriv.score_obs(model1_param).reshape( (nobs,1) )
    hess1 = model1_deriv.hessian(model1_param)
    
    
    model2_deriv = JointNormal2(yn,xn)
    ll2 = model2_deriv.loglikeobs(model2_param)
    grad2 =  model2_deriv.score_obs(model2_param).reshape( (nobs,1) )  
    hess2 = model2_deriv.hessian(model2_param)
    if return_model:
        return ll1,grad1,hess1,params1,model1_deriv,ll2,grad2,hess2,params2,model2_deriv
    return ll1,grad1,hess1,model1_param,ll2,grad2,hess2,model2_param

def gen_data(beta= 1.5, nobs=1000):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([beta,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
print(grad1.shape,hess1.shape)

(1000, 1) (1, 1)


# Ex 0

In [3]:
beta = 0

In [4]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 0.93   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.07   \\
\hline
\end{tabular}


llr:11.964985792034032, std:14.812805706405651
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.93, 0.  , 0.07]), 11.964985792034032, 14.812805706405651)


In [5]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 0.93   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.07   \\
\hline
\end{tabular}


llr:10.938029606123743, std:13.882112776178756
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.93, 0.  , 0.07]), 10.938029606123743, 13.882112776178756)


In [6]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 0.88   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.12   \\
\hline
\end{tabular}


llr:13.21449947001443, std:18.444143258948955
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.88, 0.  , 0.12]), 13.21449947001443, 18.444143258948955)


# Ex 1

In [7]:
beta = 1

In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.94 & 1.00 & 1.00 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.06 & 0.00 & 0.00 & 0.01   \\
\hline
\end{tabular}


llr:8.105813560160879, std:80.80301554670088
(array([0.94, 0.  , 0.06]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.99, 0.  , 0.01]), 8.105813560160879, 80.80301554670088)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.86 & 1.00 & 1.00 & 0.97   \\
Model 1 & 0.04 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.10 & 0.00 & 0.00 & 0.03   \\
\hline
\end{tabular}


llr:3.928195632839141, std:132.27784607732895
(array([0.86, 0.04, 0.1 ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.97, 0.  , 0.03]), 3.928195632839141, 132.27784607732895)


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.97 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.03 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}


llr:29.017056512445187, std:149.71862043117662
(array([0.97, 0.  , 0.03]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), 29.017056512445187, 149.71862043117662)


# Ex 2

In [11]:
beta = 1.5

In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.95 & 1.00 & 1.00 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.05 & 0.00 & 0.00 & 0.02   \\
\hline
\end{tabular}


llr:-2.2561635383449805, std:111.44553128718462
(array([0.95, 0.  , 0.05]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.98, 0.  , 0.02]), -2.2561635383449805, 111.44553128718462)


In [13]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.91 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.02 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.07 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}


llr:-0.3872782636335856, std:163.70024189884464
(array([0.91, 0.02, 0.07]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -0.3872782636335856, 163.70024189884464)


In [14]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.91 & 1.00 & 1.00 & 0.99   \\
Model 1 & 0.01 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.08 & 0.00 & 0.00 & 0.01   \\
\hline
\end{tabular}


llr:-56.54497515781677, std:254.58569605196155
(array([0.91, 0.01, 0.08]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.99, 0.  , 0.01]), -56.54497515781677, 254.58569605196155)


# Ex 3

In [15]:
beta = 2

In [16]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.94 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.03 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.03 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}


llr:26.332021534521196, std:172.46182590578883
(array([0.94, 0.03, 0.03]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), 26.332021534521196, 172.46182590578883)


In [17]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.96 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.02 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.02 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}


llr:51.97533337125933, std:223.36285745437416
(array([0.96, 0.02, 0.02]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), 51.97533337125933, 223.36285745437416)


In [18]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.96 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.01 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.03 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}


llr:17.785289103229253, std:296.45988840500695
(array([0.96, 0.01, 0.03]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), 17.785289103229253, 296.45988840500695)
